## Check GPS Synchronization

In [ ]:
from TimeTagger import *
from scipy.interpolate import CubicSpline
import numpy as np
import matplotlib.pyplot as plt
import datetime
from pylab import *
def log_handler(level, msg):
    if level >= LOGGER_WARNING:
        print(msg)
def time_data_aq(click_channel, pause_time, begin_channel,  N):
    # activate trivial custom logging
    setLogger(log_handler)
    # create a timetagger instance
    tagger = createTimeTagger()
    tagger.reset()
    
    data = CountBetweenMarkers(tagger, click_channel, begin_channel, n_values= N)
    pause(pause_time)
    count = data.getData()
    binwidths = data.getBinWidths()
    freeTimeTagger(tagger)
    
    return count, binwidths

In [ ]:
%matplotlib widget

In [ ]:
click_channel = 1
pause_time = 200
begin_channel = 3
N = pause_time

count, binwidths = time_data_aq(click_channel, pause_time, begin_channel, N)

In [ ]:
with open('GPS_FPGA_PLL_Integrated_120523_RX.npy', 'wb') as f:
        np.save(f, count, allow_pickle=True)

In [ ]:
x1 = np.load('GPS_FPGA_PLL_Integrated_120523_RX.npy')[:-1]
# x2 = np.load('GPS_FPGA_PLL_Integrated_112823_2.npy')[:-1]
# x2 = np.load('GPS_FPGA_PLL_2.npy')[1:]
# x3 = np.load('GPS_FPGA_PLL_3.npy')

F = []
divisor = 1

for i in range(int(len(x1)/divisor)):
    F.append(np.sum(x1[i*divisor:i*divisor + divisor], dtype=np.uint64))

# for i in range(len(count)):
#     F.append(count[i]/10)
    
fig = plt.figure(figsize=(10,5))
# F = np.array(F)
# plt.plot(F-25e6)
plt.plot(x1-25e6)
# plt.plot(x2-25e6)
plt.ylabel('Error (Hz)', fontsize = 22)
plt.xlabel('Time (s)', fontsize = 22)
# plt.ylim(-5, 5)
# plt.xlim(0,8)
plt.show()

## Check TX Data probabilities

In [ ]:
import numpy as np
test_array = np.zeros((1600))

dec_array = []

for j in range(1,2,1):
    
    file = open("C:\\Users\\sanchez-rosales.1la\\Desktop\\TX\\FPGA_Files\\FPGA Project with GPS\\SD_Data\\software\\data_folder\\data_"+str(j)+".bin", "rb");
    buff = file.read(65536*16*32)
    for i in range(16384*16*32):
        x = buff[4*i:4*i + 4]
        dec_array.append(format(int.from_bytes(x, "little"),'#032b'))
      
# diff = np.diff(dec_array)
# empty_array = np.where(diff != 1)

# n_banks =[]
# for n in range(len(empty_array[0])):
#     index = empty_array[0][n]
#     n_banks.append(((dec_array[index + 1]-dec_array[index] -1)/16384)-1)

# print(empty_array)
# print(n_banks)

In [ ]:
R = 0
L = 0
H = 0 
r = 0
l = 0
h = 0
v = 0 
for i in range(len(dec_array)-1):
    for j in range(10):
        check = dec_array[i+1][3*j + 2:3*j + 5]
        if check == '001':
            R += 1
        elif check == '010':
            L += 1
        elif check == '011':
            H += 1
        elif check == '100':
            r += 1
        elif check == '101':
            l += 1
        elif check == '110':
            h += 1
        elif check == '111':
            v += 1

total = R + L + H + r + l + h + v
Rfrac = R/total
Lfrac = L/total
Hfrac = H/total
rfrac = r/total
lfrac = l/total
hfrac = h/total
vfrac = v/total

In [ ]:
print(Rfrac)
print(Lfrac)
print(Hfrac)

print("###########################################")

print(rfrac)
print(lfrac)
print(hfrac)
print(vfrac)

print("###########################################")

print(Rfrac+Lfrac+Hfrac)
print(rfrac+lfrac+hfrac+ vfrac)

## Check RX data

In [ ]:
import numpy as np

t_0 = np.array([]);
t_1 = np.array([]);
t_2 = np.array([]);
t_3 = np.array([]);

file_size = 16*200 # 16 banks make 1MB and *32 makes it 32MB

# file = open("C:\\Users\\danis\\Desktop\\SD_data\\software\\data_folder\\data_14.bin", "rb");
file = open("C:\\Users\\danis\\Downloads\\data_0.bin", "rb");
buff = file.read(65536*file_size + file_size*4)
s = np.frombuffer( buff, dtype=np.dtype('<u4') );
for i in range(file_size):
    detector_id = s[16384*(i+1) + i]
    if detector_id == 0:
        t_0 = np.append( t_0, s[16384*i + i:16384*i + i + 16384] )
        #t_0 = np.append(t_0, t_0[-1]+51)
    elif detector_id == 1:
        t_1 = np.append( t_1, s[16384*i + i:16384*i + i + 16384] )
        #t_1 = np.append(t_1, t_1[-1]+51)
    elif detector_id == 2:
        t_2 = np.append( t_2, s[16384*i + i:16384*i + i + 16384] )
        #t_2 = np.append(t_2, t_2[-1]+51)
    elif detector_id == 3:
        t_3 = np.append( t_3, s[16384*i + i:16384*i + i + 16384] )
        #t_3 = np.append(t_3, t_3[-1]+51)

print(t_0[:20])
print(t_1[:20])
print(t_2[:20])
print(t_3[:20])

del s
del buff

In [ ]:
diffs_0 = np.diff(t_0[1:])
diffs_1 = np.diff(t_1[1:])
diffs_2 = np.diff(t_2[1:])
diffs_3 = np.diff(t_3[1:])

array_0 = np.where(diffs_0 <= 0)
array_1 = np.where(diffs_1 <= 0)
array_2 = np.where(diffs_2 <= 0)
array_3 = np.where(diffs_3 <= 0)

# array_0 = np.where(diffs_0 != 1)
# array_1 = np.where(diffs_1 != 1)
# array_2 = np.where(diffs_2 != 1)
# array_3 = np.where(diffs_3 != 1)

print(array_0)
print(array_1)
print(array_2)
print(array_3)
print(len(array_0[0]), len(array_1[0]), len(array_2[0]), len(array_3[0]))
print(np.max(diffs_0), np.max(diffs_1), np.max(diffs_2), np.max(diffs_3))
print(np.min(diffs_0), np.min(diffs_1), np.min(diffs_2), np.min(diffs_3))

print('###################################################################')

if len(array_0[0]) > 0:
    for i in range(len(array_0[0])):
        diffs_0 = np.delete(diffs_0, np.where(diffs_0 == np.min(diffs_0)))
        
if len(array_1[0]) > 0:
    for i in range(len(array_1[0])):
        diffs_1 = np.delete(diffs_1, np.where(diffs_1 == np.min(diffs_1))) 
        
if len(array_2[0]) > 0:
    for i in range(len(array_2[0])):
        diffs_2 = np.delete(diffs_2, np.where(diffs_2 == np.min(diffs_2)))
        
if len(array_3[0]) > 0:
    for i in range(len(array_3[0])):
        diffs_3 = np.delete(diffs_3, np.where(diffs_3 == np.min(diffs_3)))

array_0_2 = np.where(diffs_0 <= 0)
array_1_2 = np.where(diffs_1 <= 0)
array_2_2 = np.where(diffs_2 <= 0)
array_3_2 = np.where(diffs_3 <= 0)

print(len(array_0_2[0]), len(array_1_2[0]), len(array_2_2[0]), len(array_3_2[0]))
print(np.max(diffs_0), np.max(diffs_1), np.max(diffs_2), np.max(diffs_3))
print(np.min(diffs_0), np.min(diffs_1), np.min(diffs_2), np.min(diffs_3))